In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import keras
import keras.utils
import albumentations as A
import os

from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop,Adam

from sklearn.model_selection import train_test_split

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

In [2]:
train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

In [3]:
X_train, X_test, y_train, y_test = \
    train_test_split(images, labels, test_size=0.1, random_state=10, stratify=labels)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1000).batch(32)

print ('Obserwacje treningowe: %d\nObserwacje testowe: %d' % (X_train.shape[0], X_test.shape[0]))

Obserwacje treningowe: 54000
Obserwacje testowe: 6000


In [4]:
# Utworzenie modelu
f_mnist_model = tf.keras.Sequential([
    layers.Flatten(),
    layers.Dense(300, activation='relu'),
    layers.Dense(150, activation='relu'),
    layers.Dense(10, activation='softmax')
])
f_mnist_model

In [5]:
# Kompilacja modelu
f_mnist_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Trenowanie modelu
train_stats = f_mnist_model.fit(train_ds, epochs=20, verbose=1)

Epoch 1/20
1380/1688 [=======================>......] - ETA: 0s - loss: 0.5041 - accuracy: 0.8176

In [ ]:
f_mnist_model.summary()

In [ ]:
!mkdir -p saved_model
f_mnist_model.save('saved_model/model_fashion')

In [ ]:
plt.figure()
plt.imshow(X_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Wizualizacja w zestawie treningowym
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.xlabel(y_train[i])
plt.show()

In [ ]:
# Ocena modelu - dane testowe
loss, accuracy = f_mnist_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
# Wynik - wizualizacja
pd.DataFrame(train_stats.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

Metryka accuracy rośnie z kolejnymi epokami, natomiast błąd loss maleje.

In [ ]:
# Prognozowanie danych
y_pred = f_mnist_model.predict(X_test)
print("probs : ", y_pred[2])
print("klasa :", np.argmax(y_pred[2]))
print("rzeczywista klasa: ", y_test[2])

In [ ]:
probability_model = tf.keras.Sequential([f_mnist_model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(images)

10 liczb, które przedstawiają prawodpodobieństwo dopasowania obrazu do typów garderoby (labels).

In [ ]:
predictions[0]

Klasa (etykieta) z najwyższą ufnością

In [ ]:
np.argmax(predictions[0])

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], labels, images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], labels)
plt.tight_layout()
plt.show()

Model można wykorzystać do przewidywania obrazów. Etykiety prawidłowych prognoz są niebieskie, a nieprawidłowych czerwone.

Próba zwiększenia wartości metryki accuracy do około 97% poprzez użycie Albumentations 

In [ ]:
def albumentation(data):
    
    fliped_images = []
    
    for image in data:
        
        # Declare an augmentation pipeline
        transform = A.Compose([
        A.HorizontalFlip(p=1),
        A.GaussNoise(var_limit=(0.0001, 0.017), always_apply=True)
        ])
        
        transformed = transform(image=image)
        new_image = transformed['image']
        fliped_images.append(new_image)
        
        
        
    images_2 = np.array(fliped_images)
    
    return images_2

In [ ]:
train_2, test_2 = tf.keras.datasets.fashion_mnist.load_data()
images_2, labels_2 = train_2
images_2 = images_2/255.0
labels_2 = labels_2.astype(np.int32)

In [ ]:
new_images = albumentation(images_2)

In [ ]:
np.shape(new_images)

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(new_images, labels_2, test_size=0.1, random_state=10, stratify=labels_2)

In [ ]:
train_ds_2 = tf.data.Dataset.from_tensor_slices((X_train_2, y_train_2))

In [ ]:
train_ds_2 = train_ds_2.shuffle(1000).batch(32)

In [ ]:
train_stats_2 = f_mnist_model.fit(train_ds_2, epochs=20, verbose=1)